## College Basketball Winning Percentage Predictor

### In this machine learning notebook, we will try to predict the winning percentage of basketball teams based on various statistics about performance (other than, of course, the number of games won and the number of games played)

In [1]:
import sklearn
import pandas as pd
import numpy as np

### Importing data 

Download the dataset for the 2019 season here: 'https://www.kaggle.com/andrewsundberg/college-basketball-dataset'

In [2]:
bball_data = pd.read_csv("college-basketball-dataset/cbb19.csv")

In [3]:
bball_data

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED
0,Gonzaga,WCC,37,33,123.4,89.9,0.9744,59.0,44.2,14.9,...,35.3,25.9,61.4,43.4,36.3,30.4,72.0,7.0,E8,1.0
1,Virginia,ACC,38,35,123.0,89.9,0.9736,55.2,44.7,14.7,...,29.1,26.3,52.5,45.7,39.5,28.9,60.7,11.1,Champions,1.0
2,Duke,ACC,38,32,118.9,89.2,0.9646,53.6,45.0,17.5,...,33.2,24.0,58.0,45.0,30.8,29.9,73.6,11.2,E8,1.0
3,North Carolina,ACC,36,29,120.1,91.4,0.9582,52.9,48.9,17.2,...,30.2,28.4,52.1,47.9,36.2,33.5,76.0,10.0,S16,1.0
4,Michigan,B10,37,30,114.6,85.6,0.9665,51.6,44.1,13.9,...,27.5,24.1,51.8,44.3,34.2,29.1,65.9,9.2,S16,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,Alcorn St.,SWAC,27,10,89.0,112.6,0.0628,45.7,52.7,24.1,...,30.5,36.5,45.0,55.3,31.3,32.1,67.1,-16.7,NaN,NaN
349,New Hampshire,AE,27,5,83.7,106.1,0.0613,44.0,51.5,18.4,...,21.9,38.0,39.4,52.1,32.6,33.6,67.1,-20.2,NaN,NaN
350,Chicago St.,WAC,30,3,88.5,117.3,0.0380,44.2,57.8,22.5,...,33.1,33.9,43.5,57.9,30.7,38.5,71.9,-20.9,NaN,NaN
351,Delaware St.,MEAC,29,6,84.3,112.2,0.0358,40.0,52.4,19.0,...,25.5,39.2,37.7,52.6,29.0,34.7,71.6,-21.7,NaN,NaN


Creating a winning percentage category

In [4]:
bball_data["winning_percentage"] = bball_data["W"]/bball_data["G"]

In [5]:
bball_data["winning_percentage"]

0      0.891892
1      0.921053
2      0.842105
3      0.805556
4      0.810811
         ...   
348    0.370370
349    0.185185
350    0.100000
351    0.206897
352    0.233333
Name: winning_percentage, Length: 353, dtype: float64

In [6]:
bball_data["winning_percentage"].sort_values(ascending=False)

25     0.937500
1      0.921053
22     0.914286
47     0.909091
48     0.903226
         ...   
346    0.156250
343    0.142857
338    0.133333
263    0.129032
350    0.100000
Name: winning_percentage, Length: 353, dtype: float64

As we can see, some lower ranked teams have a very high winning percentage.  The 48th ranked team has over a 90% winning percentage, one of the best in the NCAA!  This is why straight winning percentage isn't the best metric to use for March Madness predicting!

### Creating a test set and training set

In [7]:
#creating categories based on winning percentage
#category 1 teams have a winning percentage of 0-0.20, category 2 teams are 0.20-0.30, category 3 teams are 0.30-0.40, etc.

bball_data["winning_cat"] = pd.cut(bball_data["winning_percentage"],bins=[0., .20, .30, .40, .50, .60, .70, .80, .90, 1.00],
                               labels=[1, 2, 3, 4, 5, 6, 7, 8, 9])

In [8]:
bball_data["winning_cat"]

0      8
1      9
2      8
3      8
4      8
      ..
348    3
349    1
350    1
351    2
352    2
Name: winning_cat, Length: 353, dtype: category
Categories (9, int64): [1 < 2 < 3 < 4 ... 6 < 7 < 8 < 9]

In [9]:
#splitting the data into test set and training set using stratified random sampling

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(bball_data, bball_data["winning_cat"]):
    strat_train_set = bball_data.loc[train_index]
    strat_test_set = bball_data.loc[test_index]

In [10]:
strat_train_set["winning_cat"].value_counts()/len(strat_train_set)

5    0.234043
4    0.180851
6    0.163121
3    0.156028
7    0.106383
2    0.060284
8    0.053191
1    0.031915
9    0.014184
Name: winning_cat, dtype: float64

In [11]:
strat_test_set["winning_cat"].value_counts()/len(strat_test_set)

5    0.239437
4    0.183099
6    0.169014
3    0.154930
7    0.098592
8    0.056338
2    0.056338
1    0.028169
9    0.014085
Name: winning_cat, dtype: float64

As desired, we have a test dataset and a training dataset with nearly equal proportions of winning percentage categories

In [12]:
#restore the dataset by removing overall_cat
for set_ in (strat_train_set, strat_test_set):
    set_.drop("winning_cat", axis=1, inplace=True)

In [13]:
bball_data = strat_train_set.copy()

### Interpreting the Data and Looking for Correlations

In [14]:
bball_data.corr()["winning_percentage"].sort_values(ascending=False)

winning_percentage    1.000000
W                     0.977317
WAB                   0.860602
BARTHAG               0.747883
ADJOE                 0.708684
EFG_O                 0.592872
2P_O                  0.571187
G                     0.516157
3P_O                  0.372715
ORB                   0.316523
TORD                  0.220766
FTR                   0.136341
ADJ_T                -0.043340
FTRD                 -0.199056
DRB                  -0.297171
TOR                  -0.437187
SEED                 -0.439704
3P_D                 -0.520244
2P_D                 -0.542703
EFG_D                -0.629310
ADJDE                -0.638652
Name: winning_percentage, dtype: float64

Let's choose some objective attributes that are highly correlated with winning percentage.

In [15]:
attributes = ["ADJOE", "EFG_O", "2P_O", "3P_O", "ORB", "TORD", "FTRD", "DRB", "3P_D", "2P_D", "EFG_D", "ADJDE"]

ADJOE is the points scored per 100 possessions, EFG_O is the field goal percentage, 2P_O is the two point shot percentage, ORB is the offensive rebound percentage, TORD is the steal rate, FTRD is the free throw rate allowed, DRB is the defensive rebound percentage, 2PD is the percentage of two point shots allowed, EFG_D is the percentage of field goals allowed, and ADJDE is the points allower per 100 possessions 

### Preparing the Data

In [16]:
bball_data = strat_train_set.drop("winning_percentage", axis=1)
bball_labels = strat_train_set["winning_percentage"].copy()

The pipeline below will scale all the features appropriately and fill in missing values in any data column with the median of that column

In [17]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

bball_data_final = num_pipeline.fit_transform(bball_data[attributes])

In [18]:
bball_data_final

array([[ 0.39004207,  0.95899066,  1.00384013, ...,  0.15753046,
        -0.73887622, -0.33021249],
       [ 1.8119792 ,  0.74886217,  0.44768509, ...,  0.9864433 ,
         0.23657381, -0.42024142],
       [ 0.44749408,  1.02903349,  0.66396761, ...,  0.71013902,
         0.96816133,  0.84016359],
       ...,
       [ 1.19437014,  1.23916198,  2.08525272, ..., -0.70208287,
        -0.66920122, -0.78035714],
       [-1.20425107, -1.24735848, -1.28257505, ...,  0.37243379,
         0.75913633,  1.23028895],
       [ 0.28950106,  0.46869085,  0.72576261, ...,  0.34173331,
         0.41076132,  0.39001894]])

### Finding the Best Model using Cross Validation

In [19]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

lin_reg = LinearRegression()
lin_reg.fit(bball_data_final, bball_labels)

tree_reg = DecisionTreeRegressor()
tree_reg.fit(bball_data_final, bball_labels)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [20]:
import numpy as np
from sklearn.metrics import mean_squared_error

Decision Tree Model

In [21]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, bball_data_final, bball_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [22]:
def display_scores(scores):
    print("scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(tree_rmse_scores)

scores: [0.15469167 0.15953725 0.17069052 0.13496131 0.13677666 0.11812306
 0.12677388 0.13124601 0.16260268 0.12524544]
Mean: 0.14206484718907783
Standard deviation: 0.017295493160528325


The Decision Tree Model is typically off by nearly 14%.  Not bad but let's test a few other models.

In [23]:
lin_scores = cross_val_score(lin_reg, bball_data_final, bball_labels, scoring = "neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

scores: [0.07766965 0.09000036 0.0832739  0.08009102 0.07654271 0.06365736
 0.07131574 0.09397711 0.08231685 0.06193465]
Mean: 0.07807793524806485
Standard deviation: 0.009801818289967863


Wow, the linear model was off by only 7.8%!  Let's try one last model

In [24]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(bball_data_final, bball_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [25]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, bball_data_final, bball_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

scores: [0.08469616 0.10129147 0.11201104 0.1015472  0.10088692 0.07997675
 0.10467496 0.10164631 0.09066318 0.06990288]
Mean: 0.0947296869240008
Standard deviation: 0.012345313736983707


This is good but still not quite as good as the linear model, so let's go with the linear model

### Fine-tuning our Model

We will search for the best combination of hyperparameters for our model

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'copy_X': [True], 'fit_intercept': [True]},
    {'n_jobs': [None], 'normalize': [False]},
  ]

lin_reg = LinearRegression()

grid_search = GridSearchCV(lin_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(bball_data_final, bball_labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='warn', n_jobs=None,
             param_grid=[{'copy_X': [True], 'fit_intercept': [True]},
                         {'n_jobs': [None], 'normalize': [False]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [27]:
grid_search.best_params_

{'copy_X': True, 'fit_intercept': True}

In [28]:
grid_search.best_estimator_

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.07908428570561785 {'copy_X': True, 'fit_intercept': True}
0.07908428570561785 {'n_jobs': None, 'normalize': False}


Now we know the best model to use with the appropriate hyperparameters.

### Evaluating the model with the test dataset

In [30]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("winning_percentage", axis=1)
y_test = strat_test_set["winning_percentage"].copy()

X_test_prepared = num_pipeline.fit_transform(X_test[attributes])

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)  

In [31]:
final_rmse

0.08318810336457823

In [32]:
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
loc=squared_errors.mean(),
scale=stats.sem(squared_errors)))

array([0.07031772, 0.09431828])

The 95% confidence interval of error on the test dataset is 7%-9.4%.  Not too shabby!

NOTE: Some code is used from the book, Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow